In [1]:
from dataclasses import dataclass
from typing import Union, Any

Level = Union[int, str]

class RichTypeError(TypeError):
    pass

@dataclass(frozen=True, slots=True)
class Arg:
    name: str

@dataclass(frozen=True, slots=True)
class Val:
    value: Any

def do_shit(level):
    raise RichTypeError(
        "Expected", Arg("level"), "to be", Level, "given", type(level),
        Val(level)
    )

In [2]:
from numbers import Number
from typing import Annotated, Optional, get_args, get_origin, get_type_hints
from splatlog.lib.typeguard import satisfies
from typeguard import check_type
import dataclasses

@dataclasses.dataclass(frozen=True)
class ValueRange:
    min: Optional[Number] = None
    max: Optional[Number] = None
    
    def is_valid(self, value: Number) -> bool:
        if self.min is not None and value < self.min:
            return False
        if self.max is not None and value > self.max:
            return False
        return True

Verbosity = Annotated[int, ValueRange(min=0)]

args = get_args(Verbosity)

def my_check_type(name, value, expected_type):
    check_type(name, value, expected_type)

    if get_origin(expected_type) is Annotated:
        for arg in get_args(expected_type):
            if isinstance(arg, ValueRange) and not arg.is_valid(value):
                raise TypeError(
                    "type of {} must be {}; got {} instead".format(
                        name,
                        expected_type,
                        type(value)
                    )
                )



my_check_type("verbosity", -1, Verbosity)

TypeError: type of verbosity must be typing.Annotated[int, ValueRange(min=0, max=None)]; got <class 'int'> instead

In [ ]:
from collections import defaultdict

def tree_default_factory():
    return defaultdict(tree_default_factory)

def tree_build(entries):
    tree = defaultdict(tree_default_factory)
    for (*key_path, final_key), value in entries:
        target = tree
        for key in key_path:
            target = target[key]
        target[final_key] = value
    return tree

tree_build((
    (("a", "b"), 1),
    (("a", "c"), 2),
    (("a", "d", "e"), 3),
))

defaultdict(<function __main__.tree_default_factory()>,
            {'a': defaultdict(<function __main__.tree_default_factory()>,
                         {'b': 1,
                          'c': 2,
                          'd': defaultdict(<function __main__.tree_default_factory()>,
                                      {'e': 3})})})

In [1]:
from itertools import chain
from pprint import pprint

from splatlog.lib.validate.failures import Failures, FailureGroup, format_failures

def p(name, *failures):
    print("\n# ", name)
    print(format_failures(*failures))

def failures(id, count):
    return Failures(f"Failure {id}.{n + 1}" for n in range(count))

def group(id, *failures):
    return FailureGroup(f"Group {id}", *failures)

fg_a = group("A", failures(1, 3), failures(2, 2))
fg_b = group("B", failures(3, 4), failures(4, 1))

p("Groups A and B", fg_a, fg_b)

fg_c = group("C", fg_a, fg_b, failures(5, 3))

p("Group C", fg_c)

fg_e1 = group("E1", failures(1, 0), failures(2, 0))

p("Group E1", fg_e1)

print("E1 is empty", fg_e1.is_empty())



NameError: name 'TFailure' is not defined

In [2]:
from typing import Protocol, TypeVar, runtime_checkable

T = TypeVar("T")

@runtime_checkable
class Indexed(Protocol[T]):
    def __getitem__(self, index: int) -> T:
        ...


class Blah:
    def __getitem__(self, index: int):
        return 123


b = Blah()

isinstance(b, Indexed)
isinstance([], Indexed)


True

In [2]:
from splatlog.lib.validate import *
from sys import stdout
from pathlib import Path

def p(failures):
    format_failure_into(failures, stdout)

v = validate_attr(
    "name",
    validate(lambda v: v == "c", "must be 'c'")
)

r = v(Path("/a/b/d"))

p(r)

 -    `.name`
     -    must be 'c'
